In [1]:
from pathlib import Path
import altair as alt
import duckdb
from utils import (
    loads_cae_db,
    executes_query_duckdb
)

In [2]:
cae_path = Path(
    "../../kids_chances/data/raw/cae_history.txt"
)

destionation_db = Path("../www/data/cae_db.duckdb")
cae = loads_cae_db(cae_path, destionation_db)

In [3]:
query = """
select distinct(tipo_beneficiario) from cae limit 10000"""
executes_query_duckdb(query, cae)

,tipo_beneficiario
0,NUEVO BENEFICIARIO
1,SIN CREDITO POR NO MATRICULA O POR SUSPENSION
2,BENEFICIARIO RENOVANTE
3,DESERCION
4,EGRESO
5,GRATUIDAD


# Big Table with Summary

In [4]:
query_summary = """

SELECT id_2023 as id,
año_licitacion as año_solicitud,
        genero_registrado as genero,
        dependencia_colegio as colegio,
        region_grupo_familiar as region,
        quintil_ingreso_grupo_familiar as quintil,
        COUNT(CASE WHEN tipo_beneficiario = 'NUEVO BENEFICIARIO' THEN tipo_beneficiario END)
            as total_prestamos,
        COUNT(DISTINCT(nombre_carrera || '-' || nombre_ies)) as cantidad_carreras,
        COUNT(CASE WHEN arancel_solicitado > 0 then arancel_solicitado END)
            as años_financiados,
        COUNT(DISTINCT(CASE WHEN tipo_beneficiario = 'EGRESO' then (tipo_beneficiario || nombre_carrera || nombre_ies) END))
            as total_egresos,
        COUNT(DISTINCT(CASE WHEN tipo_beneficiario = 'DESERCION' then (tipo_beneficiario || nombre_carrera || nombre_ies) END))
            as total_deserciones,
        ROUND(SUM(total_prestado_ajustado),0) as total_prestado,
        ROUND(AVG(porcentaje_financiado),1) as porcentaje_financiado

FROM cae
GROUP BY id, genero, colegio, region, quintil, año_solicitud
"""

In [5]:
final_table_to_query = executes_query_duckdb(query_summary, cae)

In [6]:
final_table_to_query

,id,año_solicitud,genero,colegio,region,quintil,total_prestamos,cantidad_carreras,años_financiados,total_egresos,total_deserciones,total_prestado,porcentaje_financiado
0,724.699,2013,F,Municipal,REGION METROPOLITANA,quintil 1 y 2,1,2,6,1,0,13579123.0,0.5
1,380.733,2013,M,Municipal,REGION METROPOLITANA,quintil 1 y 2,1,1,2,1,0,3806320.0,0.9
2,505.693,2012,M,Particular subvencionado,REGION METROPOLITANA,quintil 1 y 2,1,1,2,1,0,3481687.0,0.9
3,21.641,2013,M,Particular subvencionado,REGION DEL MAULE,quintil 1 y 2,1,1,7,1,0,20271604.0,0.5
4,60.493,2013,F,Corporación municipal,REGION METROPOLITANA,quintil 5,1,1,2,1,0,3218553.0,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1290988,1.240.230,2024,F,Particular Subvencionado,REGION METROPOLITANA,quintil 1 y 2,1,1,1,0,0,2376000.0,0.8
1290989,1.200.946,2024,M,Particular Pagado,REGION DE LOS LAGOS,quintil 1 y 2,1,1,1,0,0,2846000.0,0.3
1290990,1.241.909,2024,M,Municipal DAEM,REGION DEL LIB. B. O'HIGGINS,quintil 5,1,1,1,0,0,6424646.0,1.0
1290991,1.215.941,2024,F,Particular Pagado,REGION METROPOLITANA,quintil 5,1,1,1,0,0,6343794.0,1.0


---

---

# Aggregated Total Borrowed

In [ ]:
DOLAR = 950 # 1 dollar is $950 CLP

query_total_borrowed = f"""
Select 
    region_grupo_familiar as region,
    quintil_ingreso_grupo_familiar as quintil,
    genero_registrado as genero,
    año_operacion as año_financiado,
    round(SUM(total_prestado_ajustado)/{DOLAR}, 0) as total_prestado
FROM cae
GROUP BY region, quintil, genero, año_financiado
"""

total_borrowed = executes_query_duckdb(query_total_borrowed, cae)

mapping = {'REGION METROPOLITANA': 13,
           'REGION DEL BIO-BIO': 8,
           'REGION DEL MAULE': 7,
           'REGION DE VALPARAISO': 5,
           'REGION DE LOS RIOS': 14,
           'REGION DE COQUIMBO': 4,
           "REGION DEL LIB. B. O'HIGGINS": 6,
           'REGION DE LOS LAGOS': 10,
           'REGION DE LA ARAUCANIA': 9,
           'REGION DE ATACAMA': 3,
           'REGION DE TARAPACA': 1,
           'REGION DE ARICA Y PARINACOTA': 15,
           'REGION DE ÑUBLE': 16,
           'REGION DE ANTOFAGASTA': 2,
           'REGION DE MAGALLANES Y LA ANTARTICA': 12,
           'REGION DE AYSEN DEL G. C. IBAÑEZ': 11}

total_borrowed["region"] = total_borrowed["region"].map(mapping)

total_borrowed.to_json("../data/total_borrowed.json",
                        orient="records",
                        lines=False,
                        force_ascii=False,
                        indent=4)

total_borrowed

,region,quintil,genero,año_financiado,total_prestado
0,REGION DE VALPARAISO,quintil 4,M,2016,9389309.0
1,REGION METROPOLITANA,quintil 4,M,2016,47104187.0
2,REGION DEL BIO-BIO,quintil 3,M,2017,7320566.0
3,REGION DEL MAULE,quintil 1 y 2,M,2016,9055110.0
4,REGION DEL MAULE,quintil 1 y 2,F,2017,9419766.0
...,...,...,...,...,...
2427,REGION DE ÑUBLE,quintil 5,M,2007,52451.0
2428,REGION DE TARAPACA,quintil 4,F,2006,71681.0
2429,REGION DE MAGALLANES Y LA ANTARTICA,quintil 1 y 2,M,2006,32937.0
2430,REGION DE MAGALLANES Y LA ANTARTICA,quintil 5,M,2009,19460.0


---

---

# **Total CAES Granted**

In [8]:
query_total_granted = f"""
Select 
    region_grupo_familiar as region,
    quintil_ingreso_grupo_familiar as quintil,
    genero_registrado as genero,
    año_operacion as año_financiado,
    COUNT(tipo_beneficiario) as prestamos_entregados
FROM cae
WHERE tipo_beneficiario = 'NUEVO BENEFICIARIO'
GROUP BY region, quintil, genero, año_financiado
"""

total_granted = executes_query_duckdb(query_total_granted, cae)
total_granted

# total_granted.to_json("../data/total_cae_granted.json",
#                         orient="records",
#                         lines=False,
#                         force_ascii=False,
#                         indent=4)

,region,quintil,genero,año_financiado,prestamos_entregados
0,REGION DE VALPARAISO,quintil 4,M,2019,510
1,REGION DE LOS LAGOS,quintil 5,F,2019,528
2,REGION DE VALPARAISO,quintil 5,M,2019,1326
3,REGION METROPOLITANA,quintil 4,M,2019,2187
4,REGION DE MAGALLANES Y LA ANTARTICA,quintil 4,F,2019,52
...,...,...,...,...,...
2299,REGION DE ATACAMA,quintil 3,F,2006,7
2300,REGION DE ARICA Y PARINACOTA,quintil 3,M,2020,53
2301,REGION DE MAGALLANES Y LA ANTARTICA,quintil 4,M,2006,14
2302,REGION DE AYSEN DEL G. C. IBAÑEZ,quintil 3,M,2008,13


---

---

# **AVG LOAN**

In [9]:
DOLAR = 950 # 1 dollar is $950 CLP

query_avg_loan = f"""
WITH money_per_loan as (
    Select 
        id_2023 as id,
        año_licitacion as año_entrega,
        region_grupo_familiar as region,
        quintil_ingreso_grupo_familiar as quintil,
        genero_registrado as genero,
        ROUND(SUM(total_prestado_ajustado)/{DOLAR},0) as total_prestado,
    FROM cae
    GROUP BY id, region, quintil, genero, año_entrega
)

Select 
    region,
    quintil,
    genero,
    año_entrega,
    ROUND(AVG(total_prestado),0) as prestamo_promedio,
    COUNT(total_prestado) as prestamos_entregados

FROM money_per_loan
GROUP BY region, quintil, genero, año_entrega
"""

avg_loan = executes_query_duckdb(query_avg_loan, cae)

# avg_loan.to_json("../data/avg_loan.json",
#                          orient="records",
#                          lines=False,
#                          force_ascii=False,
#                          indent=4)

avg_loan

,region,quintil,genero,año_entrega,prestamo_promedio,prestamos_entregados
0,REGION METROPOLITANA,quintil 4,M,2011,12508.0,3846
1,REGION METROPOLITANA,quintil 1 y 2,M,2008,10915.0,4675
2,REGION DE VALPARAISO,quintil 3,M,2010,10949.0,766
3,REGION METROPOLITANA,quintil 1 y 2,F,2008,10293.0,6906
4,REGION DE LOS LAGOS,quintil 4,M,2009,12772.0,216
...,...,...,...,...,...,...
2356,REGION DE AYSEN DEL G. C. IBAÑEZ,quintil 3,M,2022,5126.0,8
2357,REGION DE COQUIMBO,quintil 5,F,2008,8952.0,1
2358,REGION DEL MAULE,quintil 5,M,2009,13831.0,4
2359,REGION DE LA ARAUCANIA,quintil 5,F,2008,17003.0,2


---

---

# **FINANCED YEARS**

In [10]:
query_financed_years = f"""
WITH prov_years_financed as (
    Select 
        id_2023 as id,
        año_licitacion as año_entrega,
        region_grupo_familiar as region,
        quintil_ingreso_grupo_familiar as quintil,
        genero_registrado as genero,
        COUNT(total_prestado_ajustado) as años_financiados
    FROM cae
    WHERE total_prestado_ajustado > 0
    GROUP BY id, region, quintil, genero, año_entrega
)

Select 
    region,
    quintil,
    genero,
    año_entrega,
    ROUND(AVG(años_financiados),2) as años_financiados,
    COUNT(años_financiados) as prestamos_entregados

FROM prov_years_financed
GROUP BY region, quintil, genero, año_entrega
"""

financed_years = executes_query_duckdb(query_financed_years, cae)

# financed_years.to_json("../www/data/financed_years.json",
#                           orient="records",
#                           lines=False,
#                           force_ascii=False,
#                           indent=4)

financed_years

,region,quintil,genero,año_entrega,años_financiados,prestamos_entregados
0,REGION DE MAGALLANES Y LA ANTARTICA,quintil 1 y 2,M,2013,3.29,80
1,REGION METROPOLITANA,quintil 1 y 2,M,2015,3.15,7292
2,REGION METROPOLITANA,quintil 1 y 2,F,2011,3.93,14195
3,REGION METROPOLITANA,quintil 4,F,2015,3.64,3914
4,REGION METROPOLITANA,quintil 3,M,2013,3.81,4026
...,...,...,...,...,...,...
2356,REGION DE VALPARAISO,quintil 5,F,2009,6.00,3
2357,REGION DE TARAPACA,quintil 5,M,2006,5.38,8
2358,REGION DE ATACAMA,quintil 3,M,2006,4.00,1
2359,REGION DE LA ARAUCANIA,quintil 5,F,2010,4.00,2


---

---

# **Percentage Financed**

In [11]:
query_percent_fin = f"""
WITH prov_percent_financed as (
    Select 
        id_2023 as id,
        año_licitacion as año_entrega,
        region_grupo_familiar as region,
        quintil_ingreso_grupo_familiar as quintil,
        genero_registrado as genero,
        AVG(porcentaje_financiado) as porcentaje_financiado
    FROM cae
    WHERE total_prestado_ajustado > 0
    GROUP BY id, region, quintil, genero, año_entrega
)

Select 
    region,
    quintil,
    genero,
    año_entrega,
    ROUND(AVG(porcentaje_financiado),2) as porcentaje_financiado,
    COUNT(porcentaje_financiado) as prestamos_entregados

FROM prov_percent_financed
GROUP BY region, quintil, genero, año_entrega
"""

percent_fin = executes_query_duckdb(query_percent_fin, cae)

percent_fin.to_json("../www/data/percent_financed.json",
                           orient="records",
                           lines=False,
                           force_ascii=False,
                           indent=4)

percent_fin

,region,quintil,genero,año_entrega,porcentaje_financiado,prestamos_entregados
0,REGION DE VALPARAISO,quintil 4,M,2011,0.89,741
1,REGION METROPOLITANA,quintil 4,M,2011,0.93,3833
2,REGION DE VALPARAISO,quintil 1 y 2,F,2010,0.91,3877
3,REGION DE LOS LAGOS,quintil 1 y 2,M,2012,0.85,864
4,REGION DE LOS LAGOS,quintil 3,F,2014,0.82,391
...,...,...,...,...,...,...
2356,REGION DEL BIO-BIO,quintil 5,F,2010,0.90,1
2357,REGION DE ANTOFAGASTA,quintil 5,M,2010,1.00,1
2358,REGION DE LOS RIOS,quintil 5,F,2009,0.89,2
2359,REGION DE LOS LAGOS,quintil 5,F,2009,0.87,1


---

---

# **Path of each student**

In [12]:
query_path = """
WITH student_career as (
    Select 
        id_2023 as id,
        año_licitacion as año_entrega,
        region_grupo_familiar as region,
        quintil_ingreso_grupo_familiar as quintil,
        genero_registrado as genero,
        (nombre_carrera || ' - ' || nombre_ies || ' - ' || tipo_ies) as id_carrera,
        COUNT(CASE WHEN tipo_beneficiario = 'NUEVO BENEFICIARIO' then tipo_beneficiario END)
            as nuevo_beneficiario_años,
        COUNT(CASE WHEN tipo_beneficiario = 'BENEFICIARIO RENOVANTE' then tipo_beneficiario END)
            as renovante_años,
        COUNT(CASE WHEN tipo_beneficiario = 'SIN CREDITO POR NO MATRICULA O POR SUSPENSION' then tipo_beneficiario END)
            as suspendido_años,
        COUNT(CASE WHEN tipo_beneficiario = 'GRATUIDAD' then tipo_beneficiario END)
            as gratuidad_años,
        MIN(CASE WHEN tipo_beneficiario = 'EGRESO' then año_operacion END)
            as año_egreso,
        MIN(CASE WHEN tipo_beneficiario = 'DESERCION' then año_operacion END)
            as año_desercion,
        COUNT(CASE WHEN arancel_solicitado > 0 then arancel_solicitado END)
            as años_financiados_cae,
        MIN(CASE WHEN cambio_ies_carrera = 'SI' then año_operacion END)
            as año_nueva_carrera


    FROM cae
    GROUP BY id,
            año_entrega,
            region,
            quintil,
            genero,
            id_carrera
    ),

    ids_count as (
        SELECT id, COUNT(id) as total_records
        FROM student_career
        GROUP BY id),

    ids_clean as (
        SELECT id
        FROM ids_count
        WHERE total_records < 3
        ),
    final as (
        SELECT sc.*
        FROM student_career sc
        INNER JOIN ids_clean ids
        ON sc.id = ids.id
        )
    
    SELECT *
    FROM final
"""
hh = executes_query_duckdb(query_path, cae)

hh

,id,año_entrega,region,quintil,genero,id_carrera,nuevo_beneficiario_años,renovante_años,suspendido_años,gratuidad_años,año_egreso,año_desercion,años_financiados_cae,año_nueva_carrera
0,954.291,2016,REGION METROPOLITANA,quintil 1 y 2,F,FONOAUDIOLOGIA - UNIVERSIDAD SAN SEBASTIÁN - U...,1,0,0,0,<NA>,<NA>,1,<NA>
1,808.347,2016,REGION DEL BIO-BIO,quintil 3,M,TECNICO EN MANTENIMIENTO INDUSTRIAL - INSTITUT...,1,1,0,0,2018,<NA>,2,<NA>
2,30.161,2009,REGION METROPOLITANA,quintil 1 y 2,M,PROGRAMA ESPECIAL DE TITULACION EN ARQUITECTUR...,0,0,2,0,<NA>,2013,0,<NA>
3,340.260,2016,REGION DEL BIO-BIO,quintil 3,F,PSICOLOGIA EXECUTIVE - UNIVERSIDAD DE LAS AMÉR...,1,4,0,0,2021,<NA>,5,<NA>
4,980.887,2016,REGION DEL MAULE,quintil 1 y 2,F,TECNICO EN EDUCACION PARVULARIA - CENTRO DE FO...,1,2,0,0,2019,<NA>,2,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448707,656.450,2016,REGION METROPOLITANA,quintil 1 y 2,F,TECNICO EN GEOLOGIA - INSTITUTO PROFESIONAL LO...,1,2,0,0,2019,<NA>,3,<NA>
1448708,548.034,2015,REGION DEL BIO-BIO,quintil 5,F,ENFERMERIA - UNIVERSIDAD SAN SEBASTIÁN - U. PR...,1,1,0,0,2017,<NA>,2,<NA>
1448709,1.099.002,2020,REGION DEL BIO-BIO,quintil 1 y 2,F,DERECHO - UNIVERSIDAD SAN SEBASTIÁN - U. PRIVA...,1,4,0,0,<NA>,<NA>,5,<NA>
1448710,1.116.267,2020,REGION METROPOLITANA,quintil 4,M,BACHILLERATO DE COMERCIAL - UNIVERSIDAD ADOLFO...,1,0,0,0,<NA>,<NA>,0,<NA>


---

---

In [13]:
h = executes_query_duckdb("select * from cae where id_2023 = '99.321' limit 100000", cae)
h

,id_2023,año_licitacion,año_operacion,genero_registrado,nacionalidad,region_grupo_familiar,quintil_ingreso_grupo_familiar,cambio_ies_carrera,tipo_beneficiario,nivel_estudio_al_ingreso,...,duracion_formal_carrera,contador_años_financiados,dependencia_colegio,arancel_solicitado,arancel_referencia,tipo_titulo,jornada,banco_administrador,porcentaje_financiado,total_prestado_ajustado
0,99.321,2009,2009,M,Chilena,REGION METROPOLITANA,quintil 1 y 2,SI,BENEFICIARIO RENOVANTE,PRIMER AÑO,...,4,1,Estudiante de Colegio o Liceo Fiscal o Municipal,950000,950000,Profesional,Vespertina,SCOTIABANK,1.000000,1750850.000
1,99.321,2006,2008,M,Chilena,REGION METROPOLITANA,quintil 1 y 2,NO,BENEFICIARIO RENOVANTE,PRIMER AÑO,...,3,0,Estudiante de Colegio o Liceo Fiscal o Municipal,430000,860000,Técnico,Diurna,SCOTIABANK,0.500000,781310.000
2,99.321,2009,2011,M,Chilena,REGION METROPOLITANA,quintil 1 y 2,NO,SIN CREDITO POR NO MATRICULA O POR SUSPENSION,PRIMER AÑO,...,2,0,Estudiante de Colegio o Liceo Fiscal o Municipal,<NA>,1111950,Técnico,Diurna,SCOTIABANK,NaN,NaN
3,99.321,2009,2018,M,Chilena,REGION METROPOLITANA,quintil 1 y 2,SI,SIN CREDITO POR NO MATRICULA O POR SUSPENSION,PRIMER AÑO,...,5,0,Estudiante de Colegio o Liceo Fiscal o Municipal,0,2328048,Profesional con Licenciatura,Vespertina,SCOTIABANK,0.000000,0.000
4,99.321,2009,2019,M,Chilena,REGION METROPOLITANA,quintil 1 y 2,SI,SIN CREDITO POR NO MATRICULA O POR SUSPENSION,PRIMER AÑO,...,3,0,Estudiante de Colegio o Liceo Fiscal o Municipal,0,1166238,Técnico,Vespertina,SCOTIABANK,0.000000,0.000
5,99.321,2009,2021,M,Chilena,REGION METROPOLITANA,quintil 1 y 2,SI,SIN CREDITO POR NO MATRICULA O POR SUSPENSION,PRIMER AÑO,...,5,0,Estudiante de Colegio o Liceo Fiscal o Municipal,0,3052978,Profesional con Licenciatura,Diurna,SCOTIABANK,0.000000,0.000
6,99.321,2009,2022,M,Chilena,REGION METROPOLITANA,quintil 1 y 2,SI,SIN CREDITO POR NO MATRICULA O POR SUSPENSION,PRIMER AÑO,...,5,0,Estudiante de Colegio o Liceo Fiscal o Municipal,0,3209520,Profesional con Licenciatura,Diurna,SCOTIABANK,0.000000,0.000
7,99.321,2009,2010,M,Chilena,REGION METROPOLITANA,quintil 1 y 2,SI,SIN CREDITO POR NO MATRICULA O POR SUSPENSION,PRIMER AÑO,...,2,0,Estudiante de Colegio o Liceo Fiscal o Municipal,<NA>,1075602,Técnico,Diurna,SCOTIABANK,NaN,NaN
8,99.321,2006,2006,M,Chilena,REGION METROPOLITANA,quintil 1 y 2,NO,NUEVO BENEFICIARIO,PRIMER AÑO,...,3,0,Estudiante de Colegio o Liceo Fiscal o Municipal,740000,740000,Técnico,Diurna,SCOTIABANK,1.000000,1552520.000
9,99.321,2009,2017,M,Chilena,REGION METROPOLITANA,quintil 1 y 2,SI,GRATUIDAD,PRIMER AÑO,...,3,0,Estudiante de Colegio o Liceo Fiscal o Municipal,0,1254993,Técnico,Vespertina,SCOTIABANK,0.000000,0.000


In [ ]:
final_table_to_query.to_parquet("../data/summary_table.parquet")